<a href="https://colab.research.google.com/github/oxayavongsa/aai-590-capstone-mental-health/blob/main/mental_health_risk_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mental Health Risk Predictor (Front-End Demo)

Demo only - not for production

In [1]:
#!pip install gradio openai joblib

In [2]:
import gradio as gr
import joblib
import numpy as np
import pandas as pd
import openai
import os

# Load trained model pipeline
model = joblib.load("logistic_pipeline_model.pkl")

# Set your OpenAI key
client = openai.OpenAI(api_key="ADD YOUR OPEN AI API KEY HERE")

In [3]:
# Extract the first step (PolynomialFeatures) from the pipeline
poly = model.named_steps['poly']

# Get input feature names (before polynomial expansion)
base_feature_names = poly.feature_names_in_.tolist()

# Get expanded feature names after PolynomialFeatures
expanded_feature_names = poly.get_feature_names_out().tolist()

base_feature_names

['Gender',
 'family_history',
 'treatment',
 'Growing_Stress',
 'Mental_Health_History',
 'Mood_Swings',
 'Coping_Struggles',
 'Work_Interest']

In [4]:
# 🔮 GPT-generated encouragement message using OpenAI v1+
def generate_response(risk, recommendation):
    try:
        prompt = (
            f"You were assessed with a {risk} level of mental health risk. "
            f"Recommendation: {recommendation}\n"
            f"Write a warm, emotionally intelligent, and uplifting message to the user."
        )

        response = client.chat.completions.create(
            model="gpt-4",  # Change to "gpt-3.5-turbo" if needed
            messages=[{"role": "user", "content": prompt}],
            temperature=0.75,
            max_tokens=150
        )

        return response.choices[0].message.content

    except Exception as e:
        return f"Error generating message: {str(e)}"

# 🧠 Main function for prediction
def predict_risk(
    diagnosis, interest_level, stress_level, swings, coping, treatment, family, gender
):
    try:
        # Map categorical answers to model-ready numbers
        interest_map = {
            "Very low": 1,
            "Somewhat low": 3,
            "Neutral": 5,
            "Somewhat high": 7,
            "Very high": 9
        }
        stress_map = {
            "No increase": 1,
            "Slight increase": 3,
            "Moderate increase": 5,
            "Significant increase": 7,
            "Extreme increase": 9
        }

        feature_values = [
            {"Female": 0, "Male": 1, "Other": 2}[gender],           # Gender
            1 if family == "Yes" else 0,                             # family_history
            1 if treatment == "Yes" else 0,                          # treatment
            stress_map[stress_level],                                # Growing_Stress
            1 if diagnosis == "Yes" else 0,                          # Mental_Health_History
            1 if swings == "Yes" else 0,                             # Mood_Swings
            1 if coping == "Yes" else 0,                             # Coping_Struggles
            interest_map[interest_level]                             # Work_Interest
        ]

        # Use exact feature names from model training
        column_names = [
            'Gender',
            'family_history',
            'treatment',
            'Growing_Stress',
            'Mental_Health_History',
            'Mood_Swings',
            'Coping_Struggles',
            'Work_Interest'
        ]

        input_df = pd.DataFrame([feature_values], columns=column_names)

        # Get prediction from model
        prediction = model.predict(input_df)[0]
        label_map = {0: "Moderate Risk", 1: "Low Risk", 2: "High Risk"}
        risk_label = label_map[prediction]

        recommendation_map = {
            "Moderate Risk": "Try to maintain balance and consider periodic self-assessments or speaking with a counselor.",
            "Low Risk": "You appear emotionally stable. Keep up healthy habits and self-care.",
            "High Risk": "We encourage reaching out to a licensed mental health professional for support."
        }

        recommendation = recommendation_map[risk_label]
        gpt_message = generate_response(risk_label, recommendation)

        return risk_label, recommendation, gpt_message

    except Exception as e:
        return "Error", "There was a problem processing your input.", str(e)

In [5]:
# 🎨 Gradio App UI
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## 🧠 Mental Health Risk Predictor")
    gr.Markdown("Answer a few short questions to receive your mental health risk level, a helpful recommendation, and an AI-generated message of encouragement.")

    with gr.Row():
        diagnosis = gr.Radio(["Yes", "No"], label="Have you ever been diagnosed with a mental health condition?")
        interest_level = gr.Radio(
            ["Very low", "Somewhat low", "Neutral", "Somewhat high", "Very high"],
            label="How interested are you in your work or daily activities?"
        )
        stress_level = gr.Radio(
            ["No increase", "Slight increase", "Moderate increase", "Significant increase", "Extreme increase"],
            label="How much has your stress level increased recently?"
        )

    with gr.Row():
        swings = gr.Radio(["Yes", "No"], label="Have you been experiencing frequent mood swings?")
        coping = gr.Radio(["Yes", "No"], label="Are you struggling to cope with daily challenges?")
        treatment = gr.Radio(["Yes", "No"], label="Are you currently receiving mental health treatment?")

    with gr.Row():
        family = gr.Radio(["Yes", "No"], label="Does your family have a history of mental health issues?")
        gender = gr.Radio(["Female", "Male", "Other"], label="What is your gender?")

    submit_btn = gr.Button("Assess My Risk")

    risk_output = gr.Textbox(label="🧭 Your Risk Level")
    reco_output = gr.Textbox(label="📌 Recommendation")
    gpt_output = gr.Textbox(label="💬 AI-Powered Encouragement", lines=5)

    submit_btn.click(
        fn=predict_risk,
        inputs=[diagnosis, interest_level, stress_level, swings, coping, treatment, family, gender],
        outputs=[risk_output, reco_output, gpt_output]
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b4679ae1afd1716590.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
